# Model Registry 최신 모델을 이용한 모델 평가하기

## 1. 사전 준비 과정

In [ ]:
import boto3
import json
from sagemaker import get_execution_role
from time import strftime
import calendar
import time

In [ ]:
iam_client = boto3.client('iam')
role=get_execution_role()
base_role_name=role.split('/')[-1]

In [ ]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']

### 1.1 MLOps에서 활용할 Policy 설정하기

해당 HOL에서 구현할 아키텍처에 필요한 managed policy를 아래와 같이 정의합니다. Role을 별도 생성하셔도 되지만 HOL의 편의성을 위해 SageMaker Notebook/Studio와 동일한 Role에 policy를 추가하여 계속 활용합니다.

In [ ]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSCodeCommitFullAccess'
)

## 2. SageMaker Studio의 설정값에서 model_package_group_name 생성/가져오기

In [ ]:
sm_client = boto3.client("sagemaker")

model_package_groups_list = sm_client.list_model_package_groups(
        SortBy='CreationTime',
        SortOrder='Descending')

if len(model_package_groups_list['ModelPackageGroupSummaryList']) == 0:
    print("Creating a model package group for a new model")

    import time
    model_package_group_name = "yolov5"
    model_package_group_input_dict = {
     "ModelPackageGroupName" : model_package_group_name,
     "ModelPackageGroupDescription" : "Sample model package group"
    }

    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))
else:    
    model_package_group_name= model_package_groups_list['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
model_package_group_name

In [ ]:
# sm_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

## 3. Create Amazon EventBridge Rule

model registry에서 모델이 **Approved**되었을 때 이벤트 트리거를 만들기 위한 설정을 Amazon EventBridge Rule을 이용하여 설정합니다.

In [ ]:
event_client = boto3.client('events')

In [ ]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [ ]:
rule_name = 'yolov5_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the yolov5 model',
)

## 4. 모델 평가를 위한 Lambda function 생성

EventBridge 에서 Rule 만족하는 이벤트가 발생했을 때 실행되는 Lambda Function을 정의합니다. Lambda Function 은 테스트 데이터를 예측하는 Batch transform job을 수행하게 됩니다.


In [ ]:
%store -r

In [ ]:
lambda_client = boto3.client('lambda')

In [ ]:
lambda_trust_policy=json.dumps({
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
})

In [ ]:
role_name='lambda-assume-role_'+ strftime("%m%d-%H%M%s")
try:
    for role_list in iam_client.list_roles()['Roles']:
        pre_role_name = role_list['RoleName']
        if pre_role_name.split("_")[0] in ['lambda-assume-role']:
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
            )
            iam_client.detach_role_policy(
                RoleName=pre_role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
            )
            iam_client.delete_role(RoleName=pre_role_name)
except:
    pass
finally:
    lambda_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=lambda_trust_policy
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite'
    )
    time.sleep(10)
role_name

In [ ]:
# The name of our algorithm
algo_name = "yolov5-batch-inference"

In [ ]:
# Get the region defined in the current configuration (default to us-west-2 if none defined)
my_session = boto3.session.Session()
region = my_session.region_name

repo_name=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{algo_name}:1.0"

In [ ]:
%%capture
!sh build_and_push.sh $account_id $region $algo_name $repo_name

In [ ]:
# codecommit_client = boto3.client("codecommit")

# repositoryName = codecommit_client.list_repositories()['repositories'][0]['repositoryName']
# codecommit_repo = codecommit_client.get_repository(repositoryName=repositoryName)['repositoryMetadata']['cloneUrlHttp']
codecommit_repo = 'https://git-codecommit.XXXXXXXXX.amazonaws.com/v1/repos/XXXXXXXXX'  ## <== CodeCommit 이름 추가

In [ ]:
%store -r
print(f"bucket : {bucket} \n sec_arn : {sec_arn} \ncodecommit_repo : {codecommit_repo}")

In [ ]:
lambda_name='yolov5-batch-inference'
try:
    lambda_client.delete_function(FunctionName=lambda_name)
except:
    pass
finally:
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_name,
        Role=lambda_role['Role']['Arn'],
        Code={
            'ImageUri': repo_name
        },
        PackageType='Image',
        Description='Create the latest version-based yolov5 model',
        Timeout=600,
        MemorySize=512,
        Environment={
          'Variables': {
              "bucket_name" : bucket,
              "model_package_group_name" : model_package_group_name,
              "sec_arn" : sec_arn,
              "role" : role,
              "instanace_type" : "ml.g4dn.xlarge",
              "instanace_count" : "1",
              "region_name" : region,
              "codecommit_repo" : codecommit_repo
              
              
          }
      }
    )

In [ ]:
# response = lambda_client.update_function_code(
#     FunctionName=lambda_name,
#     ImageUri=repo_name
# )

In [ ]:
lambda_permission_response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='InvokeLambdaFunction',
    Action='lambda:InvokeFunction',
    Principal="events.amazonaws.com",
    SourceArn=event_rule['RuleArn'],
)

Amazon EventBridge에 위에서 생성한 Lambda function을 타켓으로 설정합니다.

In [ ]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': lambda_response['FunctionArn']
        }
    ]
)

## 5. TEST - Approval the latest Model Version

In [ ]:
print(f"artifacts_dir : {artifacts_dir}, model_package_group_name : {model_package_group_name}")

In [ ]:
latest_modelpkg_list = sm_client.list_model_packages( ModelPackageGroupName=model_package_group_name,
                                                      ModelApprovalStatus='PendingManualApproval',
                                                      SortBy='CreationTime',
                                                      SortOrder='Descending')

In [ ]:
ModelPackageArn = latest_modelpkg_list['ModelPackageSummaryList'][0]['ModelPackageArn']
ModelPackageArn

#### - 최신 배포된 모델에 대해 승인 수행

In [ ]:
model_package_update_input_dict = {
    "ModelPackageArn" : ModelPackageArn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

#### - Model Version이 필요한 경우 가상으로 추가하는 명령어

In [ ]:
# modelpackage_inference_specification =  {
#     "InferenceSpecification": {
#       "Containers": [
#          {
#             "Image": inf_image_uri,
#          }
#       ],
#       "SupportedContentTypes": [ "application/x-image" ],
#       "SupportedResponseMIMETypes": [ "application/x-image" ],
#    }
#  }
# # Specify the model source
# model_url = f"{artifacts_dir}model.tar.gz"

# # Specify the model data
# modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

# create_model_package_input_dict = {
#     "ModelPackageGroupName" : model_package_group_name,
#     "ModelPackageDescription" : "Model to detect 3 different types ('Platelets', 'RBC', 'WBC')",
#     "ModelApprovalStatus" : "PendingManualApproval"
# }
# create_model_package_input_dict.update(modelpackage_inference_specification)

# create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
# model_package_arn = create_mode_package_response["ModelPackageArn"]
# print('ModelPackage Version ARN : {}'.format(model_package_arn))